# HOMER DATA PROCESSING SCRIPT
The goal of this script is to take the output file of HOMER Pro Batch Runs (.xlsx file containing one optimal case/sheet) and produce a condensed data set + plots.

General flow:
1) Run HOMER study via batch process. Select "export results" and name the file.


2) Import file to this script.


3) Use Pandas to pull .xlsx data into a more usable format. 


4) Use collected data to build plots. 

Goals:


Have this script take a large HOMER study (>100 cases) and build one easily accessible database.


Potentially couple this script with a HOMER input reading script to do cartography w/HOMER geography + results.


In [2]:
# Block of dependencies and working directory
import matplotlib.pyplot as plt
import conda
import numpy  as np 
import pandas as pd
import os 
import shutil
directory = 'd:/repo/homer/ClimateStudies/OutputProcessing/Data' #home PC
#directory = 'c:/repo/homer/ClimateStudies/OutputProcessing/Data' #laptop

## Following this guide for xlsx reading
https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook

In [3]:
# reads, maps all sheets 
files = os.listdir(directory)
output_df = {} #creates empty dictionary
names = [] #creates empty list for future name storage
for file in files:
    if file.endswith('.xlsx'):
        xls = pd.ExcelFile(directory + '/' + file) #reads file
        names = names + xls.sheet_names
        for sheet_name in xls.sheet_names: #pulls individual sheets
            output_df[sheet_name] = xls.parse(sheet_name) #puts each sheet into a dictionary w/key "sheet_name"
#display(output_df[sheet_name])

In [31]:
#pulling interesting data points from each 
opt_out = pd.DataFrame(columns=['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)','GenMin'])
new_cases = pd.DataFrame(columns=['FileName','Cost/COE\n($)','Architecture/GenLarge\n(kW)'])
n = len(names)
for i in range(n):
    name = names[i] # Grabs Sheet Name
    KCC = name[-3:]     #grabs KCC from last 3 values
    if KCC[-2:].casefold() == ('Af'.casefold()):
            KCC = 'Af'
            Location = name[5:-2]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                case_num = name[:6] # Grabs Case Number 
            else:
                case_num = name[:5]
    elif KCC[-2:].casefold() == 'Am'.casefold():
            KCC = 'Am'
            Location = name[5:-2]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                case_num = name[:6] # Grabs Case Number 
            else:
                case_num = name[:5]
    else: 
            Location = name[5:-3]
            if name[:6] == 'Case25' or name[:6] == 'Case35':
                case_num = name[:6] # Grabs Case Number 
            else:
                case_num = name[:5]
    KCC = KCC.lower()
    filtered_out = output_df[name].filter(['Architecture/PV\n(kW)', 'Architecture/G1500', 'Architecture/GenLarge\n(kW)', 'Architecture/1kWh LI',
                                                 'Cost/COE\n($)']) #Filtering objects of interest
    filtered_out['CaseName'] = Location #adding actual case name to series
    filtered_out['CaseNum'] = case_num # adding case num to series
    filtered_out['KCC'] = KCC
    framed = filtered_out.iloc[[0]] #making the actual df
    framed.set_index(['CaseNum','CaseName'], inplace=True, drop=True) #building df w/ casename and num
    opt_out = pd.concat([framed,opt_out]) #concating all DFs together
    temp = pd.DataFrame(columns=['FileName','Cost/COE\n($)','Architecture/GenLarge\n(kW)'])
    if opt_out.loc[case_num].loc[Location].loc['Architecture/GenLarge\n(kW)'] >= 2000:
        #display('This is working')
        opt_out.loc[case_num].loc[Location].loc['GenMin'] = 'False' #flags case for further cost increases
        if case_num == 'Case4':
            #shutil.copyfile('d:/repo/homer/ClimateStudies/ProductionCases/Case4/' + name + '.homer', directory + '/NewCase/' + 'Case5' + name[5:] + '.homer')
            temp.loc['FileName'] = name
            temp.set_index(['FileName'])
            temp['Cost/COE\n($)'] = opt_out.loc[case_num].loc[Location].loc['Cost/COE\n($)']
            temp['Architecture/GenLarge\n(kW)'] = opt_out.loc[case_num].loc[Location].loc['Architecture/GenLarge\n(kW)']
            #display(temp)
    else: 
        opt_out.loc[case_num].loc[Location].loc['GenMin'] = 'True' #nuclear cost is "maxed out"
    new_cases = pd.concat([temp,new_cases])
    
# Data Cleanup    
opt_out = opt_out[ ['KCC'] + [ col for col in opt_out.columns if col != 'KCC' ] ] #sorting df to have KCC first
opt_out = opt_out.sort_index()
opt_out.to_excel("LocationalVarianceTrimmed.xlsx")
new_cases.to_excel("NewCaseData.xlsx")
display(opt_out)
display(new_cases)

KCC  Architecture/PV\n(kW)  Architecture/G1500  \
CaseNum CaseName                                                    
Case1   Anchorage  dfc           62704.406458                27.0   
        Ashland    dfb           40033.750925                19.0   
        AtlantaGA  cfa           93091.971417                 3.0   
        BangorME   dfb           61149.251524                22.0   
        BendOre    bsk           36672.913375                23.0   
...                ...                    ...                 ...   
Case4   Syracuse   dfb           32226.588138                 6.0   
        TexasOil   bsh           12331.998758                 7.0   
        UPMich     dfb           24484.569835                 7.0   
        VegasNV    bwh           12652.728563                 5.0   
        WadenaMN   dfa           10442.319473                 7.0   

                   Architecture/GenLarge\n(kW) Architecture/1kWh LI  \
CaseNum CaseName                                                      
Case1   Anchorage                          NaN               249345   
        Ashland                            NaN                83153   
        AtlantaGA                          NaN               121562   
        BangorME                           NaN               138129   
        BendOre                            NaN                97456   
...                                        ...                  ...   
Case4   Syracuse                        3000.0                31696   
        TexasOil                        2000.0                35767   
        UPMich                          3000.0                34867   
        VegasNV                         3000.0                30691   
        WadenaMN                        3000.0                26381   

                   Cost/COE\n($) GenMin  
CaseNum CaseName                         
Case1   Anchorage       0.439344    NaN  
        Ashland         0.207777    NaN  
        AtlantaGA       0.252569    NaN  
        BangorME        0.302428    NaN  
        BendOre         0.233693    NaN  
...                          ...    ...  
Case4   Syracuse        0.178524    NaN  
        TexasOil        0.132140    NaN  
        UPMich          0.162385    NaN  
        VegasNV         0.155194    NaN  
        WadenaMN        0.138689    NaN  

[372 rows x 7 columns]

,FileName,Cost/COE\n($),Architecture/GenLarge\n(kW)
FileName,Case4WadenaMNDfa,0.138689,3000.0
FileName,Case4VegasNVBwh,0.155194,3000.0
FileName,Case4UPMichDfb,0.162385,3000.0
FileName,Case4TexasOilBSh,0.132140,2000.0
FileName,Case4SyracuseDfb,0.178524,3000.0
...,...,...,...
FileName,Case4BangorMEDfb,0.193386,4000.0
FileName,Case4AZYumaBwh,0.159500,3000.0
FileName,Case4AtlantaGACfa,0.189311,4000.0
FileName,Case4AshlandDfb,0.171955,3000.0


In [ ]:
#for index = opt_out.ind
#display(pd.MultiIndex.get_level_values(opt_out('CaseNum')))
Case1 = opt_out.loc[("Case1",slice(None))]
Case2 = opt_out.loc[("Case2",slice(None))]
Case25 = opt_out.loc[("Case25",slice(None))]
Case3 = opt_out.loc[("Case3",slice(None))]
Case35 = opt_out.loc[("Case35",slice(None))]
Case4 = opt_out.loc[("Case4",slice(None))]
display(Case4)
#Case1.plot(y='Cost/COE\n($)', use_index=True)

In [ ]:
fig = plt.figure()
fig, ax=plt.subplots(figsize=(24,12))
ax.scatter(Case1.index.values,Case1['Cost/COE\n($)']*1000)
ax.scatter(Case1.index.values,Case2['Cost/COE\n($)']*1000)
ax.scatter(Case1.index.values,Case25['Cost/COE\n($)']*1000)
ax.scatter(Case1.index.values,Case3['Cost/COE\n($)']*1000)
ax.scatter(Case1.index.values,Case35['Cost/COE\n($)']*1000)
ax.scatter(Case1.index.values,Case4['Cost/COE\n($)']*1000)
ax.legend(['NoNuclear', 'LowCostNuclear','LoMidCostNuclear', 'MidCostNuclear', 'MidHiCostNuclear','HighCostNuclear'], fontsize='large')
plt.ylabel('Cost [\$/MWhre]')
plt.xlabel('Location')
plt.xticks(rotation=90);
ax.grid(True)


In [ ]:
m = len(Case2)
fig = plt.figure(figsize=(24,12))
barWidth = 0.25
# Grabbing Generator Size Data
bars1 = Case2["Architecture/GenLarge\n(kW)"]
bars15 = Case25["Architecture/GenLarge\n(kW)"]
bars2 = Case3["Architecture/GenLarge\n(kW)"]
bars2 = Case35["Architecture/GenLarge\n(kW)"]
bars3 = Case4["Architecture/GenLarge\n(kW)"]
#ranges
r1 = np.arange(m)
r15 = [x+barWidth for x in r1]
r2 = [x+barWidth for x in r15]
r25 = [x+barWidth for x in r2]
r3 = [x+barWidth for x in r25]

# Plotting
plt.bar(r1, bars1, width=barWidth, edgecolor='white', label='LowCostNuclear')
plt.bar(r15, bars15, width=barWidth, edgecolor='white', label='LoMidCostNuclear')
plt.bar(r2, bars2, width=barWidth, edgecolor='white', label='MidCostNuclear')
plt.bar(r25, bars25, width=barWidth, edgecolor='white', label='MidCostNuclear')
plt.bar(r3, bars3, width=barWidth, edgecolor='white', label='HighCostNuclear')
# Add xticks on the middle of the group bars
plt.xlabel('Location', fontweight='bold')
plt.ylabel('Microreactor Capacity [MW]', fontweight='bold')
plt.xticks([r + barWidth for r in range(m)], Case2.index.values)
plt.xticks(rotation=90);
plt.grid(True)
 
